In [1]:
# -*- coding: utf-8 -*-
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Perceptron
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support

In [3]:
fTrn = open("train.txt",mode='r')
fTst = open("test.txt",mode='r')

def features(sentence, tags, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        #'prev-tag':'' if index == 0 else tags[index - 1],
        #'next-tag':'' if index == len(sentence) - 1 else tags[index + 1],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'suffix-4': sentence[index][-4:],
        'suffix-5': sentence[index][-5:],
        'suffix-6': sentence[index][-6:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
    }

In [4]:
trn_X = []
tst_X = []
trn_Y = []
tst_Y = []
for i in fTrn.readlines():
	sentence = [w.split('|')[0] for w in i.split()]
	tags_  = [w.split('|')[-1].split('.')[0] for w in i.split()]
	tags = []
	for i in tags_:
		if i in ['I-NP','B-NP','O']:
			tags.append('NN')
		elif i.endswith(':?'):
			tags.append(i.split(':?')[0])
		else:
			tags.append(i)
	temp = []
	for i in range(len(sentence)):
		trn_X.append(features(sentence,tags,i))
		trn_Y.append(tags[i])

for i in fTst.readlines():
	sentence = [w.split('|')[0] for w in i.split()]
	tags_  = [w.split('|')[-1].split('.')[0] for w in i.split()]
	tags = []
	for i in tags_:
		if i in ['I-NP','B-NP','O']:
			tags.append('NN')
		elif i.endswith(':?'):
			tags.append(i.split(':?')[0])
		else:
			tags.append(i)
	temp = []
	for i in range(len(sentence)):
		tst_X.append(features(sentence,tags,i))
		tst_Y.append(tags[i])

In [55]:
vectorizer = DictVectorizer(sparse=False)
X = vectorizer.fit_transform(trn_X[:20000]+tst_X[:5000])
trnX = X[:20000]
tstX = X[20000:]

In [56]:
print tstX.shape

(5000, 21960)


In [57]:
tagLst = list(set(list(trn_Y[:20000]+tst_Y[:5000])))
dic = {}
for k,v in enumerate(tagLst):
    dic[v] = k
print dic

{'WQ': 3, 'NEG': 1, 'DEM': 2, 'JJ': 0, 'RP': 4, 'NN': 5, 'VAUX': 13, 'PRP': 7, 'RB': 8, 'NST': 9, 'NNP': 10, 'INTF': 11, 'RDP': 12, 'CC': 6, 'XC': 14, 'VM': 15, 'UNK': 16, 'SYM': 17, 'QC': 18, 'INJ': 19, 'QF': 20, 'PSP': 21, 'QO': 22}


In [58]:
print len(tagLst)

23


In [59]:
trnY = []
for i in trn_Y[:20000]:
    trnY.append(dic[i])
trnY = np.asarray(trnY)
tstY = []
for i in tst_Y[:5000]:
    tstY.append(dic[i])
tstY = np.asarray(tstY)

In [60]:
print tstY.shape

(5000,)


In [ ]:
# train model
clf = SVC()
clf.fit(trnX[:10000],trnY[:10000])
print "Accuracy : ",clf.score(tst_X[:5000],tst_Y[:5000])
print "Scores prec,rec,fm : ",precision_recall_fscore_support(y_true, y_pred, average='macro')

In [61]:
print trnX[0]

[ 0.  1.  0. ...,  0.  0.  0.]


In [62]:
trnX = np.reshape(trnX, (trnX.shape[0],1,trnX.shape[1]))
tstX = np.reshape(tstX, (tstX.shape[0],1,tstX.shape[1]))

In [63]:
tstX.shape

(5000, 1, 21960)

In [64]:
tstY.shape

(5000,)

In [66]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Flatten

model = Sequential()
model.add(LSTM(100, input_shape=(1,21960), return_sequences=True))
model.add(Flatten())
model.add(Dense(22, activation = 'softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(trnX,trnY,batch_size=128, nb_epoch=50,validation_data=(tstX, tstY))

Train on 20000 samples, validate on 5000 samples
Epoch 1/50
  128/20000 [..............................] - ETA: 16s - loss: 3.0903 - acc: 0.0625

IndexError: index 22 is out of bounds for axis 1 with size 22
Apply node that caused the error: AdvancedIncSubtensor{inplace=False,  set_instead_of_inc=True}(Alloc.0, TensorConstant{1}, ARange{dtype='int64'}.0, Elemwise{Cast{int32}}.0)
Toposort index: 141
Inputs types: [TensorType(float32, matrix), TensorType(int8, scalar), TensorType(int64, vector), TensorType(int32, vector)]
Inputs shapes: [(128, 22), (), (128,), (128,)]
Inputs strides: [(88, 4), (), (8,), (4,)]
Inputs values: ['not shown', array(1, dtype=int8), 'not shown', 'not shown']
Outputs clients: [[GpuFromHost(AdvancedIncSubtensor{inplace=False,  set_instead_of_inc=True}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/home/514cs1007/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):
  File "/home/514cs1007/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 288